In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

# Task 2: Подзапрос

А теперь давайте усложним задачу. Вы знаете, что такое подзапрос? Это как запрос, но только внутри запроса. Чем он может быть полезен? Например, с его помощью можно сделать определенную агрегацию для выбранных данных. Однако следует помнить, что вложенные запросы выполняются в первую очередь, и только после этого — основной запрос.

Что вам нужно сделать:

## 1. Создайте соединение с базой данных с помощью библиотеки sqlite3

In [3]:
import sqlite3
con = sqlite3.connect('/content/drive/MyDrive/School21/day11/data/checking-logs.sqlite')

## 2. Получите схему таблицы checker

In [4]:
pd.io.sql.read_sql("PRAGMA table_info(checker)", con)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,status,TEXT,0,None,0
2,2,success,INTEGER,0,None,0
3,3,timestamp,TIMESTAMP,0,None,0
4,4,numTrials,INTEGER,0,None,0
5,5,labname,TEXT,0,None,0
6,6,uid,TEXT,0,None,0


## 3. Получите только первые 10 строк таблицы checker, чтобы проверить, как она выглядит

In [5]:
pd.io.sql.read_sql("SELECT * from checker LIMIT 10", con)

,index,status,success,timestamp,numTrials,labname,uid
0,0,checking,0,2020-04-16 21:12:50.740474,5,None,admin_1
1,1,ready,0,2020-04-16 21:12:54.708365,5,code_rvw,admin_1
2,2,checking,0,2020-04-16 21:46:47.769088,7,None,admin_1
3,3,ready,0,2020-04-16 21:46:48.121217,7,lab02,admin_1
4,4,checking,0,2020-04-16 21:53:01.862637,6,code_rvw,admin_1
5,5,ready,0,2020-04-16 21:53:05.373389,6,code_rvw,admin_1
6,6,checking,0,2020-04-17 05:18:51.965864,1,None,None
7,7,ready,0,2020-04-17 05:19:02.744528,1,project1,user_4
8,8,checking,0,2020-04-17 05:22:35.249331,2,project1,user_4
9,9,ready,1,2020-04-17 05:22:45.549397,2,project1,user_4


## 4. Подсчитайте, сколько строк удовлетворяют следующим условиям, используя только один запрос с любым количеством подзапросов.

- Подсчитайте количество строк из таблицы pageviews, но только с пользователями из таблицы checker, у которых:

  * status = 'ready' (мы не хотим анализировать логи со статусом checking);

  * numTrials = 1 (мы хотим проанализировать только первые коммиты, поскольку только они информируют о том, когда студент начал работать над лабораторным заданием);
  * названия лабораторных заданий должны быть только из следующего списка: laba04, laba04s, laba05, laba06, laba06s, project1. Только они были активными во время проведения эксперимента.


* Сохраните их в датафрейме checkers в столбце cnt. Датафрейм будет представлять собой только одну ячейку.

In [6]:
pd.io.sql.read_sql("SELECT * from pageviews LIMIT 10", con)

,index,uid,datetime
0,0,admin_1,2020-04-17 12:01:08.463179
1,1,admin_1,2020-04-17 12:01:23.743946
2,2,admin_3,2020-04-17 12:17:39.287778
3,3,admin_3,2020-04-17 12:17:40.001768
4,4,admin_1,2020-04-17 12:27:30.646665
5,5,admin_1,2020-04-17 12:35:44.884757
6,6,admin_1,2020-04-17 12:35:52.735016
7,7,admin_3,2020-04-17 12:36:21.401412
8,8,admin_3,2020-04-17 12:36:22.023355
9,9,admin_1,2020-04-17 13:55:19.129243


In [7]:
checkers = pd.io.sql.read_sql("""
            SELECT COUNT(uid) as cnt
            FROM pageviews
            WHERE uid in 
            (SELECT uid
            FROM checker
            WHERE status = 'ready'
              and numTrials = 1 
              and labname in('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
            )
            """, con)

In [8]:
checkers

,cnt
0,985


## 5. Закройте соединение.

 

In [9]:
con.close()